<h1>Import Functions</h1>

In [57]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

<h1>Load Data</h1>

In [30]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')  

<h1>Load Column Headers</h1>

In [119]:
headers = []
i=0
for th in soup.find_all('th'):
    if i < 3:
        headers.insert(i, th.text.replace('\n','') )
    i +=1
#df = pd.DataFrame(columns=headers)
#print(df)

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


<h1>Load Data into DataFrame</h1>

In [262]:
html_rows = soup.find('tbody')
y=0
out_rows = []
for tr in html_rows.find_all('tr'):
    x=0
    html_cols = []
    for td in tr.find_all('td'):
        html_cols.insert(x,td.text.replace('\n',''))
        x +=1
    if len(html_cols) > 0:
        #check if neighbourhood not assigned
        if html_cols[2] == 'Not assigned':
            html_cols[2] = html_cols[1]
        # are we on the first record?
        if y > 0 :
            # make sure the previous record exists
            if len(out_rows[y-1]) > 0:
                # check for same postal code
                if out_rows[y-1][0] == html_cols[0]:
                    out_rows[y-1][2] = out_rows[y-1][2] + ', ' + html_cols[2]
                else:
                    out_rows.insert(y,html_cols)
                    y +=1
            else:
                out_rows.insert(y,html_cols)
                y +=1
        else:
            out_rows.insert(y,html_cols)
            y +=1
df = pd.DataFrame(out_rows)
df.columns=headers
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)
print(df.head(10))

  Postcode           Borough                     Neighbourhood
0      M3A        North York                         Parkwoods
1      M4A        North York                  Victoria Village
2      M5A  Downtown Toronto         Harbourfront, Regent Park
3      M6A        North York  Lawrence Heights, Lawrence Manor
4      M7A      Queen's Park                      Queen's Park
5      M9A         Etobicoke                  Islington Avenue
6      M1B       Scarborough                    Rouge, Malvern
7      M3B        North York                   Don Mills North
8      M4B         East York   Woodbine Gardens, Parkview Hill
9      M5B  Downtown Toronto          Ryerson, Garden District


<h1>View the DataFrame Shape</h1>

In [235]:
df.shape

(103, 3)

<h1>Get the Locations</h1>

In [255]:
locations = pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h1>Merge the Locations</h1>

In [263]:
df = df.rename(columns={'Postcode': 'Postal Code'})
df_locations = pd.merge(df, locations, how='right', on='Postal Code')
df_locations = df_locations.rename(columns={'Latitude_x': 'Latitude','Longitude_y': 'Longitude'})
df_locations.head(11)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
